In [8]:
import numpy as np
import cv2 as cv

In [82]:
#Gaussian Blur to reduce noise and detail
image_vec = cv.imread('image_0322.jpg', 1)
blurred = cv.GaussianBlur(image_vec, (5,5), 0)

In [83]:
#Edge Detection with pre-trained structured forest ML model
blurred_float = blurred.astype(np.float32)/255.0
edgeDetector = cv.ximgproc.createStructuredEdgeDetection("model.yml")
edges = edgeDetector.detectEdges(blurred_float) * 255.0
cv.imwrite('edge-raw.jpg', edges)

True

In [84]:
def filterOutSaltPepperNoise(edgeImg):
    #Get rid of salt & pepper noise
    count = 0
    lastMedian = edgeImg
    median = cv.medianBlur(edgeImg, 3)
    while not np.array_equal(lastMedian, median):
        #Get those pixels that gets zeroed out
        zeroed = np.invert(np.logical_and(median, edgeImg))
        edgeImg[zeroed] = 0
        
        count = count + 1
        if count > 50:
            break
        lastMedian = median
        median = cv.medianBlur(edgeImg, 3)

In [85]:
edges_8u = np.asarray(edges, np.uint8)
filterOutSaltPepperNoise(edges_8u)
cv.imwrite('edge.jpg', edges_8u)

True

In [86]:
def findLargestContour(edgeImg):
    contours, hierarchy = cv.findContours(
        edgeImg,
        cv.RETR_EXTERNAL,
        cv.CHAIN_APPROX_SIMPLE
    )
    #From among them, find the contours with large surface area
    contoursWithArea = []
    for contour in contours:
        area = cv.contourArea(contour)
        contoursWithArea.append([contour, area])
        
    contoursWithArea.sort(key=lambda tupl: tupl[1], reverse=True)
    largestContour = contoursWithArea[0][0]
    return largestContour

In [87]:
contour = findLargestContour(edges_8u)
#Draw the contour on the original image
contourImg = np.copy(image_vec)
cv.drawContours(contourImg, [contour], 0, (0, 255, 0), 2, cv.LINE_AA, maxLevel=1)
cv.imwrite('contour.jpg', contourImg)

True

In [88]:
mask = np.zeros_like(edges_8u)
cv.fillPoly(mask, [contour], 255)

#Calculate sure foreground area by dilating the mask 
mapFg = cv.erode(mask, np.ones((5,5), np.uint8), iterations=10)

#Mark initial mask as "probable background"
#and mapFg as sure foreground
trimap = np.copy(mask)
trimap[mask == 0] = cv.GC_BGD
trimap[mask == 255] = cv.GC_PR_BGD
trimap[mask == 255] = cv.GC_FGD

#Visualise trimap
trimap_print = np.copy(trimap)
trimap_print[trimap_print == cv.GC_PR_BGD] = 128
trimap_print[trimap_print == cv.GC_BGD] = 255
cv.imwrite('trimap.png', trimap_print)

True

In [89]:
#Run grabcut
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)
rect = (0, 0, mask.shape[0] - 1, mask.shape[1] - 1)
cv.grabCut(image_vec, trimap, rect, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_MASK)

#Create mask again
mask2 = np.where(
    (trimap == cv.GC_FGD) | (trimap == cv.GC_PR_FGD),
    255,
    0
).astype('uint8')
cv.imwrite('mask2.jpg', mask2)

True

In [90]:
contour2 = findLargestContour(mask2)
mask3 = np.zeros_like(mask2)
cv.fillPoly(mask3, [contour2], 255)

array([[  0,   0, 255, ...,   0,   0,   0],
       [  0, 255, 255, ...,   0,   0,   0],
       [  0, 255, 255, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [91]:
#Blended Alpha cut-out
mask3 = np.repeat(mask3[:, :, np.newaxis], 3, axis=2)
mask4 = cv.GaussianBlur(mask3, (3, 3), 0)
alpha = mask4.astype(float) * 1.1 #making blend stronger
alpha[mask3 > 0] = 255.0
alpha[alpha > 255] = 255.0

foreground = np.copy(image_vec).astype(float)
foreground[mask4 == 0] = 0
background = np.ones_like(foreground, dtype=float) * 255.0

cv.imwrite('foreground.png', foreground)
cv.imwrite('background.png', background)
cv.imwrite('alpha.png', alpha)

#Normalise the alpha mask to keep intensity between 0 and 1
alpha = alpha/255.0
#Multiply the foreground with the alpha matte
foreground = cv.multiply(alpha, foreground)
#Multiply the background with (1-alpha)
background = cv.multiply(1.0 - alpha, background)
#Add the masked foreground and background
cutout = cv.add(foreground, background)

cv.imwrite('cutout.jpg', cutout)

True